In [1]:
import json
import random
import os
from glob import glob

In [2]:
files = glob('/home/husein/ssd3/malaysian-ultrachat/*.jsonl')
files

['/home/husein/ssd3/malaysian-ultrachat/ultrachat-jurnaldbp.jsonl',
 '/home/husein/ssd3/malaysian-ultrachat/ultrachat-jurnaldbp-malay.jsonl',
 '/home/husein/ssd3/malaysian-ultrachat/ultrachat-gov.my.jsonl',
 '/home/husein/ssd3/malaysian-ultrachat/ultrachat-crossref-melayu.jsonl',
 '/home/husein/ssd3/malaysian-ultrachat/ultrachat-lom-agc.jsonl',
 '/home/husein/ssd3/malaysian-ultrachat/ultrachat-crossref-melayu-malay.jsonl',
 '/home/husein/ssd3/malaysian-ultrachat/ultrachat-epenerbitan-malay.jsonl',
 '/home/husein/ssd3/malaysian-ultrachat/ultrachat-ms-wikipedia.jsonl']

In [3]:
for f in files:
    new_f = f'pair-{os.path.split(f)[-1]}'
    data = []
    with open(f) as fopen:
        for l in fopen:
            data.append(json.loads(l))
    
    indices = {i for i in range(len(data))}
    pairs = []
    for i in range(len(data)):
        l = data[i]
        context = l[0]['content']
        pos = []
        for l_ in l[1:]:
            v = list(l_.values())
            v = [v_ for v_ in v if v_ is not None and v_ not in {'user', 'assistant'}]
            pos.extend(v)
        pos = list(set(pos))

        neg_i = random.choice(list(indices - {i}))
        neg = []
        for l_ in data[neg_i][1:]:
            v = list(l_.values())
            v = [v_ for v_ in v if v_ is not None and v_ not in {'user', 'assistant'}]
            neg.extend(v)

        neg = list(set(neg))

        pairs.append({
            'query': context,
            'pos': pos,
            'neg': neg
        })

        l = l[1:]
        inputs = []
        for no, r in enumerate(l):
            if r['content_ms'] is None:
                break

            inputs.append(r['content_ms'])

        if len(inputs) % 2 != 0:
            inputs = inputs[:-1]

        for k in range(0, len(inputs), 2):

            neg_i = random.choice(list(indices - {i}))

            pairs.append({
                'query': inputs[k],
                'pos': [context, inputs[k + 1]],
                'neg': [data[neg_i][0]['content']]
            })
        
        for k in range(1, len(inputs), 2):
            neg_i = random.choice(list(indices - {i}))

            pairs.append({
                'query': inputs[k],
                'pos': [context, inputs[k - 1]],
                'neg': [data[neg_i][0]['content']]
            })
            
        l = l[1:]
        inputs = []
        for no, r in enumerate(l):
            if r['content'] is None:
                break

            inputs.append(r['content'])

        if len(inputs) % 2 != 0:
            inputs = inputs[:-1]

        for k in range(0, len(inputs), 2):

            neg_i = random.choice(list(indices - {i}))

            pairs.append({
                'query': inputs[k],
                'pos': [context, inputs[k + 1]],
                'neg': [data[neg_i][0]['content']]
            })
        
        for k in range(1, len(inputs), 2):
            neg_i = random.choice(list(indices - {i}))

            pairs.append({
                'query': inputs[k],
                'pos': [context, inputs[k - 1]],
                'neg': [data[neg_i][0]['content']]
            })
            
    with open(new_f, 'w') as fopen:
        for l in pairs:
            if not len(l['pos']) or not len(l['neg']):
                continue
            fopen.write(f'{json.dumps(l)}\n')